# 感知器

## 感知器模型

- 感知器的提出，基本上提出了统一的模型
    - 分类判定模型
        - $y^{\prime} = f(xW + b)$
        
    - 损失模型：
        - $L(y, y^{\prime}) = ？$非常多
            - 均方损失函数：f(x)=x   ：线性回归
            - 对数损失函数：
                - sogmoid交叉熵 ： 逻辑回归
                - softmax交叉熵
            - hinge孙叔函数： linear SVM + kernel sVM
            - huber损失

## 感知器的实现

In [ ]:
import tensorflow as tf
import numpy as np
import sklearn.datasets  as ds

# 1. 感知器的输出模型（预测模型）
#    - 1.1. 输入数据（鸢尾花数据  100 * 4）
x = tf.placeholder(dtype=tf.float32, shape=(None, 4))    # 也可以使用constan创建常张量，也可以使用Variable创建被管理张量（张量包装器）
y = tf.placeholder(dtype=tf.float32, shape=(None))
#    - 1.2. 训练的数据（权重矩阵 + 偏置项）：可训练的参数一定使用Variable包装器管理数据与生命周期
w_v = tf.random.uniform(shape=(4, 1), minval=-0.1, maxval=0.1, dtype=np.float32)
# w_v = np.random.uniform(low=-0.1, high=0.1, size=(4, 1))
w = tf.Variable(w_v) 
# print(w)

b_v = tf.random.uniform(shape=(), minval=-0.1, maxval=0.1, dtype=np.float32)
# w_v = np.random.uniform(low=-0.1, high=0.1, size=(4, 1))
b = tf.Variable(b_v) 

#    - 1.3. 激活函数
# y_o = x @ w + b    # 内积 
y_o = tf.add(tf.matmul(x, w) , b)

# y_predict =  tf.sigmoid(y_o) # 
y_predict =  tf.sigmoid(y_o)

# 2. 优化模型
#    - 2.1. 损失函数（y , y'）
# loss = tf.reduce_mean(tf.square( tf.subtract(y_o , y)))    #  1/n * (y-y)^2

loss = tf.losses.sigmoid_cross_entropy(y, y_o)
#    - 2.2. 损失函数最小的优化（梯度下降：封装独有的函数：学习率）
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

optimizer_op = optimizer.minimize(loss)

# Graph的保存
writer = tf.summary.FileWriter('./nn', tf.get_default_graph())

writer.flush()
writer.close()
#3. 模型的执行(数据加载，格式化)
data, target = ds.load_iris(return_X_y=True)
data = data[50: 150]
target =target[0 : 100]

session =tf.Session()
session.run(tf.global_variables_initializer())
#   - 3.1. 模型的训练 
N = 10000
batch = 10
batch_size= len(data)//batch
for n in range(N):
    for i in range(batch_size):     # 随机梯度下降
        session.run(
            optimizer_op, 
            feed_dict={
                x:data[i * batch: (i+1) * batch], 
                y:target[i * batch: (i+1) * batch]
            }
        )   # 保持维数相同
    # 输出误差
    if n % 100 == 0:
        err = session.run(loss, feed_dict={x: data, y: target})
        print("损失函数输出：",  err)
                                                                                        
    
#   - 3.2. 使用模型预测（图可视化）
pre = session.run(y_predict, feed_dict={x: data})

pre[pre>0.5] = 1
pre[pre<=0.5] = 0
print(pre.shape)

(pre[:,0] == target).sum()

# 前馈神经网络

- 网络结构
    1. 输入：4
    2. 隐藏层：5    sigmoid    4 * 5
    3. 隐藏层：3    sigmoid    5 * 3
    4. 输出层:  3（3类样本：\[1,0,0\], \[0,1,0\],\[0,0,1\]）:    3 * 3

In [2]:
# modeule 与package导入
import tensorflow as tf
import sklearn.datasets as ds
import numpy as np


In [3]:
# 模型
#  ------------------
x = tf.placeholder(dtype=tf.float32, shape=(None, 4))
y = tf.placeholder(dtype=tf.float32, shape=(None, 3))  # 0 = [1,0,0]  1 =[0,1,0]  2= [0,0,1]

# -------------------

In [5]:
# 训练的变量
# 第一层的权重
w_1 = tf.Variable(tf.random.uniform(shape=(4, 5), minval=-0.01, maxval=0.01, dtype=np.float32))
b_1 = tf.Variable(tf.random.uniform(shape=(1, 5), minval=-0.01, maxval=0.01, dtype=np.float32))
# 第二层

w_2 = tf.Variable(tf.random.uniform(shape=(5, 3), minval=-0.01, maxval=0.01, dtype=np.float32))
b_2 = tf.Variable(tf.random.uniform(shape=(1, 3), minval=-0.01, maxval=0.01, dtype=np.float32))

# 第三层
w_3 = tf.Variable(tf.random.uniform(shape=(3, 3), minval=-0.01, maxval=0.01, dtype=np.float32))
b_3 = tf.Variable(tf.random.uniform(shape=(1, 3), minval=-0.01, maxval=0.01, dtype=np.float32))

In [7]:
# 向后的输出计算
# 第一层输出
yo_1 = tf.add(tf.matmul(x, w_1), b_1)
yp_1 = tf.sigmoid(yo_1)
# 第二层输出
yo_2 = tf.add(tf.matmul(yp_1, w_2), b_2)
yp_2 = tf.sigmoid(yo_2)   #tf.nn.relu(yo_12)
# 第三层输出
yo_3 = tf.add(tf.matmul(yp_2,  w_3), b_3)


yo = tf.sigmoid(yo_3)


In [9]:
#  前馈模型（损失模型，优化模型）
loss = tf.losses.sigmoid_cross_entropy(y, yo_3)    # 可以自己定义
# loss = 1.0/2 * (y- yo_3) * (y-yo_3)   均方差

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)

# 优化操作
train_op = optimizer.minimize(loss)


In [10]:
# 初始化
session = tf.Session()
session.run(tf.global_variables_initializer())

In [12]:
# 加载数据集
data,  target = ds.load_iris(return_X_y=True)
# 数据格式化
X = data
Y = np.zeros(shape=(len(target), 3), dtype=np.float32)
for i in range(len(target)):
    l_pos= target[i]
    Y[i][l_pos] = 1


In [17]:
# 训练
N = 10000
batch = 10
batch_size= len(X)//batch
for n in range(N):
    for i in range(batch_size):     # 随机梯度下降
        session.run(
            train_op, 
            feed_dict={
                x:X[i * batch: (i+1) * batch], 
                y:Y[i * batch: (i+1) * batch]
            }
        )   # 保持维数相同
    # 输出误差
    if n % 100 == 0:
        err = session.run(loss, feed_dict={x: X, y: Y})
        print("损失函数输出：",  err)

损失函数输出： 0.63651544
损失函数输出： 0.6366941
损失函数输出： 0.63668823
损失函数输出： 0.6366828
损失函数输出： 0.6366777
损失函数输出： 0.6366728
损失函数输出： 0.6366679
损失函数输出： 0.63666296
损失函数输出： 0.6366573
损失函数输出： 0.63665
损失函数输出： 0.6366375
损失函数输出： 0.63661003
损失函数输出： 0.6365384
损失函数输出： 0.6363268
损失函数输出： 0.63562113
损失函数输出： 0.63249594
损失函数输出： 0.602258
损失函数输出： 0.43252695
损失函数输出： 0.3506258
损失函数输出： 0.3299855
损失函数输出： 0.32156396
损失函数输出： 0.3165646
损失函数输出： 0.31235504
损失函数输出： 0.30702332
损失函数输出： 0.30002865
损失函数输出： 0.292916
损失函数输出： 0.28482714
损失函数输出： 0.26967207
损失函数输出： 0.2344959
损失函数输出： 0.18500867
损失函数输出： 0.14299764
损失函数输出： 0.11581217
损失函数输出： 0.099096775
损失函数输出： 0.08845298
损失函数输出： 0.081269406
损失函数输出： 0.076098576
损失函数输出： 0.07215108
损失函数输出： 0.068998866
损失函数输出： 0.066396
损失函数输出： 0.06418565
损失函数输出： 0.062261164
损失函数输出： 0.0605515
损失函数输出： 0.05901106
损失函数输出： 0.057611562
损失函数输出： 0.0563342
损失函数输出： 0.055165008
损失函数输出： 0.05409305
损失函数输出： 0.05310882
损失函数输出： 0.052204467
损失函数输出： 0.051372837
损失函数输出： 0.050607167
损失函数输出： 0.04990149
损失函数输出： 0.049250454
损失函数输出

In [18]:
# 测试
pre = session.run(
    yo, 
    feed_dict={
        x:X
    })



array([[9.9838090e-01, 1.7188787e-03, 1.1026859e-06],
       [9.9826348e-01, 1.8410683e-03, 1.1026859e-06],
       [9.9834311e-01, 1.7583072e-03, 1.1026859e-06],
       [9.9821031e-01, 1.8960238e-03, 1.1622906e-06],
       [9.9838936e-01, 1.7099380e-03, 1.0430813e-06],
       [9.9835932e-01, 1.7413199e-03, 1.1026859e-06],
       [9.9833035e-01, 1.7715991e-03, 1.1026859e-06],
       [9.9833614e-01, 1.7655790e-03, 1.1026859e-06],
       [9.9816155e-01, 1.9463599e-03, 1.1920929e-06],
       [9.9827218e-01, 1.8320680e-03, 1.1622906e-06],
       [9.9839401e-01, 1.7051101e-03, 1.0430813e-06],
       [9.9828398e-01, 1.8197298e-03, 1.1622906e-06],
       [9.9828190e-01, 1.8220246e-03, 1.1622906e-06],
       [9.9835908e-01, 1.7416179e-03, 1.1026859e-06],
       [9.9843782e-01, 1.6591549e-03, 1.0728836e-06],
       [9.9843085e-01, 1.6664565e-03, 1.1324883e-06],
       [9.9841952e-01, 1.6784370e-03, 1.0430813e-06],
       [9.9837023e-01, 1.7299950e-03, 1.1026859e-06],
       [9.9836814e-01, 1.732

In [23]:
o_pre = pre.argmax(axis=1)
(o_pre == target).sum()

147

In [24]:
session.run(w_1)

array([[-1.7447284 , -0.4268512 , -2.064081  , -0.41228718, -1.7928782 ],
       [-2.0035727 , -1.6513062 , -1.6261929 , -1.6401    , -2.0889754 ],
       [ 3.282474  ,  2.5860765 ,  3.4433851 ,  2.5407963 ,  3.3863845 ],
       [ 4.152177  ,  1.338027  ,  4.3500047 ,  1.337834  ,  4.2308455 ]],
      dtype=float32)

In [25]:
session.run(b_1)

array([[-4.719931  , -0.27585724, -4.842547  , -0.25763616, -4.804512  ]],
      dtype=float32)

In [26]:
session.run(w_2)

array([[-1.6204336, -1.6428704, -3.9158003],
       [-3.263194 , -3.5470624,  1.9251413],
       [-1.6815736, -1.7028964, -4.097348 ],
       [-3.2157123, -3.4939075,  1.8701997],
       [-1.7003876, -1.7293723, -4.022923 ]], dtype=float32)

In [27]:
# 1. 可视化（2维，网格可视化）： 分类曲线是线性还是非线性。

# 2. 把上面的数据集使用图像测试效果。

# 3. 交叉验证训练

# 4. 综合项目的数据集使用神经网络做分类

# 5. 利用sklearn的classification-report, 

# 6. AUC曲线绘制出来 ROC
147/150

0.98